# scikit-learn中的回归问题

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:
boston = datasets.load_boston()

X = boston.data
y = boston.target

X = X[y<50.0]
y = y[y<50.0]

In [3]:
X.shape

(490, 13)

In [4]:
from sklearn.model_selection._split import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

## scikit-learn中的线性回归

In [5]:
from sklearn.linear_model.base import LinearRegression

lin_reg = LinearRegression()

In [6]:
lin_reg.fit(X_train, y_train)

/usr/local/seamonster/MachineLearningClassicAlgorithmEnv/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [7]:
lin_reg.coef_

array([ -1.14235739e-01,   3.12783163e-02,  -4.30926281e-02,
        -9.16425531e-02,  -1.09940036e+01,   3.49155727e+00,
        -1.40778005e-02,  -1.06270960e+00,   2.45307516e-01,
        -1.23179738e-02,  -8.80618320e-01,   8.43243544e-03,
        -3.99667727e-01])

In [8]:
lin_reg.intercept_

32.645660839653509

In [9]:
lin_reg.score(X_test,y_test)

0.80089161995191005

### KNN Regressor （KNN解决回归问题）

In [10]:
from sklearn.neighbors.regression import KNeighborsRegressor

knn_reg = KNeighborsRegressor()

#### KNN数据归一化

In [11]:
from sklearn.preprocessing.data import StandardScaler
standard_scaler = StandardScaler()
standard_scaler.fit(X_train)
X_train_nor = standard_scaler.transform(X_train)
X_test_nor = standard_scaler.transform(X_test)

In [12]:
knn_reg.fit(X_train_nor, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [13]:
knn_reg.score(X_test_nor, y_test)

0.82230080487286983

呃，knn效果比线性回归还好。。。  
要是把网格搜索也用上岂不更离谱。。。

In [14]:
from sklearn.model_selection._search import GridSearchCV

param_grid = [
    {
        'weights':['uniform'],
        'n_neighbors':[i for i in range(1,11)]
    },
    {
        'weights':['distance'],
        'n_neighbors':[i for i in range(1,11)],
        'p':[i for i in range(1,6)]
    }
]

knn_reg2 = KNeighborsRegressor()
grid_search = GridSearchCV(knn_reg2, param_grid, n_jobs=-1, verbose=1)
grid_search.fit(X_train_nor, y_train)

Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    1.5s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'weights': ['uniform'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}, {'weights': ['distance'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

注意下面的分数与那些分类器回归器的score用的不是同一种标准，所以不能直接与它们比较

In [15]:
grid_search.best_score_

0.79480244433269864

In [16]:
grid_search.best_params_

{'n_neighbors': 5, 'p': 1, 'weights': 'distance'}

In [17]:
knn_reg_grid_search = grid_search.best_estimator_

In [18]:
knn_reg_grid_search.score(X_test_nor, y_test)

0.85652703298427613